# **2x2 큐브 퍼즐 활용 보드 게임**



2024320019 컴퓨터학과 최명국<br>
https://github.com/choimyeongguk/2by2_Cube<br>
** 파이썬 기능 검색 시 ChatGPT를 활용했으나, 프로그램 작성은 전부 직접 했음을 밝힙니다.<br>

<br>


---






In [13]:
# 필요한 라이브러리
from collections import deque
import json
import random
import os

# **1. 큐브 객체 선언**
속성(큐브의 상태) 저장 방법:<br>
상-전-우-하-후-좌 면 순으로 순번을 정한다.<br>
왼쪽 위 조각부터 시작해 시계순으로 순번을 정한다.<br>
각 순번에 따라 그 조각면의 색깔을 2차원 리스트에 저장한다.
처음부터 리스트가 아닌 숫자로 저장하면 저장공간을 절약하고 시간도 줄어든다. 하지만 이번에는 그렇게 많은 연산을 하지 않아도 되는 가벼운 프로젝트이고, 개발 상의 실수도 줄일 수 있다는 장점이 있으므로, 이차원 리스트를 활용한다.

메서드(큐브의 조작 및 처리):
 - print() : 큐브의 전개도를 출력한다.

 - input() : 사용자가 직접 큐브의 상태를 입력한다. 단, 알고리즘의 단순화를 위해 노-파-주 조각을 하-후-좌 위치에 고정한 상태로 입력하도록 한다. 3x3 큐브의 경우 축이 존재해 큐브 전체의 회전을 고려하지 않아도 되지만, 2x2 큐브는 축이 없어 큐브 자체의 회전이 가능하고, 이로인해 하나의 상태임에도 24 가지의 가능한 표현이 생길 수 있다. 이를 방지하기 위해 하나의 조각을 고정점으로 잡으면, 큐브 자체의 회전으로 인해 생기는 문제를 해결할 수 있다.
 - getNum() : 리스트 형태로 저장되어 있는 큐브의 상태를 숫자로 변환한다. 너비우선탐색에 사용될 큐와 결과가 저장될 딕셔너리의 저장용량을 줄일 수 있다. 조각은 6개의 색깔을 가질 수 있으므로, 6진법을 활용하여 리스트의 정보를 정수로 변환할 수 있다.
 - set() : 숫자 정보를 리스트 변환하여 적용한다. getNum()의 역순.
 - solved() : 큐브가 맞춰졌는지 여부를 반환한다.
 - rotate() : 큐브 자체의 회전을 구현한다.
 - U(), U_p(), F(), F_p(), R(), R_p() : 각 면 개별의 회전을 구현한다. rotate() 메서드에 각각 회전의 정보를 전달한다. 노-파-주 조각이 하-후-좌 위치에 고정되어 있기 때문에 3 개의 면의 회전 만으로 6면 전체의 회전을 대체할 수 있다. (U->D, F->B, R->L와 같이 대체할 수 있다.)

In [14]:
class Cube:
  def __init__(self):
    self.__CtoN = { 'W': 0, 'G': 1, 'R': 2, 'Y': 3, 'B': 4, 'O': 5 }
    self.__NtoC = { 0: 'W', 1: 'G', 2: 'R', 3: 'Y', 4: 'B', 5: 'O' }
    self.state = [
        [ 'W', 'W', 'W', 'W' ], # 0, White,  Up
        [ 'G', 'G', 'G', 'G' ], # 1, Green,  Front
        [ 'R', 'R', 'R', 'R' ], # 2, Red,    Right
        [ 'Y', 'Y', 'Y', 'Y' ], # 3, Yellow, Down
        [ 'B', 'B', 'B', 'B' ], # 4, Blue,   Back
        [ 'O', 'O', 'O', 'O' ]  # 5, Orange, Left
    ]

  # Print net of cube
  def print(self):
    print(f'\n    {self.state[0][0]} {self.state[0][1]}')
    print(f'    {self.state[0][3]} {self.state[0][2]}')
    for i in [ 5, 1, 2, 4 ]:
      print(f'{self.state[i][0]} {self.state[i][1]} ', end = '')
    print()
    for i in [ 5, 1, 2, 4 ]:
      print(f'{self.state[i][3]} {self.state[i][2]} ', end = '')
    print()
    print(f'    {self.state[3][0]} {self.state[3][1]}')
    print(f'    {self.state[3][3]} {self.state[3][2]}\n')
    return self

  # Input cube's color
  def input(self):
    self.state[0] = list(input('윗   면 입력 >> ').split())
    self.state[1] = list(input('앞   면 입력 >> ').split())
    self.state[2] = list(input('오른 면 입력 >> ').split())
    self.state[3] = list(input('아랫 면 입력 >> ').split())
    self.state[4] = list(input('뒷   면 입력 >> ').split())
    self.state[5] = list(input('왼   면 입력 >> ').split())
    if self.state[3][3] != 'Y' or self.state[4][2] != 'B' or self.state[5][3] != 'O':
      print('노-파-주 조각을 아래-뒤-왼쪽 자리에 위치해 주세요\n')
      self.input()
    else:
      return self

  # Convert list to integar. 120byte -> 32byte
  def getNum(self):
    ret = 0
    for i in self.state:
      for j in i:
        ret *= 6
        ret += self.__CtoN[j]
    return ret

  # Convert integar to list
  def set(self, num):
    self.state = []
    for i in range(6):
      self.state.insert(0, [])
      for j in range(4):
        self.state[0].insert(0, self.__NtoC[num % 6])
        num //= 6
    return self

  # solved -> True, not solved -> False
  def solved(self):
    if self.getNum() == 731796345686735:
      return True
    return False

  # Rotate based on args
  def rotate(self, face, clockwise, li1, li2):
    li = [ 3, 2, 1 ] if clockwise else [ 1, 2, 3 ]
    tmp = self.state[face][0]
    self.state[face][0] = self.state[face][li[0]]
    self.state[face][li[0]] = self.state[face][li[1]]
    self.state[face][li[1]] = self.state[face][li[2]]
    self.state[face][li[2]] = tmp

    for li in li2:
      tmp = self.state[li1[0]][li[0]]
      self.state[li1[0]][li[0]] = self.state[li1[1]][li[1]]
      self.state[li1[1]][li[1]] = self.state[li1[2]][li[2]]
      self.state[li1[2]][li[2]] = self.state[li1[3]][li[3]]
      self.state[li1[3]][li[3]] = tmp

  def U(self):
    self.rotate(0, True,  [ 1, 2, 4, 5 ], [[ 0, 0, 0, 0 ], [ 1, 1, 1, 1 ]])
    return self

  def U_p(self):
    self.rotate(0, False, [ 1, 5, 4, 2 ], [[ 0, 0, 0, 0 ], [ 1, 1, 1, 1 ]])
    return self

  def F(self):
    self.rotate(1, True,  [ 0, 5, 3, 2 ], [[ 2, 1, 0, 3 ], [ 3, 2, 1, 0 ]])
    return self

  def F_p(self):
    self.rotate(1, False, [ 0, 2, 3, 5 ], [[ 2, 3, 0, 1 ], [ 3, 0, 1, 2 ]])
    return self

  def R(self):
    self.rotate(2, True,  [ 0, 1, 3, 4 ], [[ 1, 1, 1, 3 ], [ 2, 2, 2, 0 ]])
    return self

  def R_p(self):
    self.rotate(2, False, [ 0, 4, 3, 1 ], [[ 1, 3, 1, 1 ], [ 2, 0, 2, 2 ]])
    return self

# **2. 가능한 경우의 수 완전 탐색**
성능을 전혀 고려하지 않고 구현에 초점을 둬서 시간이 조금 걸릴 수도 있습니다.<br>
직접 해본 결과 5분정도 실행됐습니다.<br>
시간이 부족하실 경우 첨부한 json 파일을 써서 다음 단계로 넘어가주세요.

In [15]:
queue = deque()
depth = {}   # minimun rotation to solve
path = {}   # optimal path to solve

def insert(cnt, state, rotation, num):
  if num not in depth:
    queue.append(num)
    depth[num] = depth[state] + 1
    path[num] = rotation
    cnt += 1
  return cnt

# Initialize queue for BFS and dictionary. Insert start node(solved case)
queue.append(Cube().getNum())
depth[queue[0]] = 0
path[queue[0]] = ""
cnt = 1   # num of case depth = 0

while queue:
  iter = cnt
  cnt = 0
  for i in range(iter):
    state = queue.popleft()
    cube = Cube().set(state)
    cnt = insert(cnt, state, "U'", cube.U().getNum())          # U
    cnt = insert(cnt, state, "U",  cube.U_p().U_p().getNum())  # U'
    cnt = insert(cnt, state, "F'", cube.U().F().getNum())      # F
    cnt = insert(cnt, state, "F",  cube.F_p().F_p().getNum())  # F'
    cnt = insert(cnt, state, "R'", cube.F().R().getNum())      # R
    cnt = insert(cnt, state, "R",  cube.R_p().R_p().getNum())  # R'

# Save result as JSON
# num of key:val pair is 3,674,160 & max of depth is 14 -> searched correctly!
with open('depth.json', 'w', encoding = 'utf-8') as file:
  json.dump(depth, file, indent = 4)
with open('path.json', 'w', encoding = 'utf-8') as file:
  json.dump(path, file, indent = 4)

In [16]:
# Load dictionary from JSON file
with open('depth.json', 'r', encoding = 'utf-8') as file:
  depth = json.load(file)
depth = { int(k): v for k, v in depth.items() }   # Convert key string to integar
with open('path.json', 'r', encoding = 'utf-8') as file:
  path = json.load(file)
path = { int(k): v for k, v in path.items() }

# **3. 2x2 큐브 풀이 프로그램**

예시 스크램블 : F F R' F' R' U R' F F U F F U' (13회전)<br>



```
# 입력
윗   면 입력 >> R O Y O
앞   면 입력 >> W B Y W
오른 면 입력 >> R W G G
아랫 면 입력 >> R R O Y
뒷   면 입력 >> B B B Y
왼   면 입력 >> W G G O
```




```
# 출력
    R O
    O Y
W G W B R W B B
O G W Y G G Y B
    R R
    Y O

rotation : 11
solution : R F R' F R' F R' U F R' U'
```

In [17]:
cube = Cube().input().print()
operation = { "U": lambda: cube.U(), "U'": lambda: cube.U_p(),
              "F": lambda: cube.F(), "F'": lambda: cube.F_p(),
              "R": lambda: cube.R(), "R'": lambda: cube.R_p() }
print(f'\nrotation : {depth[cube.getNum()]}')
print('solution : ', end = '')
while True:
  state = cube.getNum()
  print(path[state] + ' ', end = '')
  if depth[state] == 0:
    break
  operation[path[state]]()

윗   면 입력 >> W W Y B
앞   면 입력 >> W G Y G
오른 면 입력 >> O G B R
아랫 면 입력 >> R B W Y
뒷   면 입력 >> R O B R
왼   면 입력 >> G O Y O

    W W
    B Y
G O W G O G R O 
O Y G Y R B R B 
    R B
    Y W


rotation : 11
solution : R F' R F R' R' F U F U' U'  

# **4. 보드 게임 구현**

규칙 :
1. 2인 이상의 참여자는 게임 시작 시 N 포인트를 가지고 시작한다. (N은 임의 설정 가능)
2. 2x2 큐브를 섞는다.
3. 첫 번째 참여자부터 순서대로 차례가 진행된다.
4. 현재 순번의 참여자는 주사위를 던진 후, 나온 수만큼 큐브를 회전할 수 있다.
5. 이때, 각 회전마다 포인트를 하나씩 소모하여 찬스를 이용할 수 있다.
6. 찬스를 이용하면 현재 큐브의 상태에서 맞춰질 때까지 몇 회전 남았는 지 알 수 있다.
7. 찬스를 보고 기존의 상태로 큐브를 복구시킬 때도 회전수가 차감된다.
8. 자기 순번에서 큐브를 맞춘 참여자가 승리한다.

In [18]:
class Player():
  def __init__(self, player_number, point):
    self.num = player_number  # num of player
    self.point = point        # starting point
    self.chance = 0           # decided by dice
    self.cube = 0             # cube instance

  def operation(self, player_input):
    self.chance -= 1
    {
        "U": lambda: self.cube.U(), "U'": lambda: self.cube.U_p(),
        "F": lambda: self.cube.F(), "F'": lambda: self.cube.F_p(),
        "R": lambda: self.cube.R(), "R'": lambda: self.cube.R_p()
    }.get(player_input)()

  def play(self, cube):
    self.cube = cube.print()
    self.chance = random.randint(1, 6)
    print(f'{self.num}번 플레이어 차례 입니다.')
    print(f'주사위 눈은 {self.chance}입니다.')
    while self.chance:
      print(f'    기회가 {self.chance}번 남았습니다.')
      while True:
        player_input = input('    회전하려면 기호를, 힌트를 보려면 H를 입력하세요 >> ')
        if player_input == 'H':
          if self.point:
            print(f'        정답까지 {depth[self.cube.getNum()]} 회전 남았습니다.')
            self.point -= 1
            print(f'        포인트가 {self.point}점 남았습니다.')
          else:
            print('        포인트가 없습니다.')
        elif player_input in [ "U", "U'", "F", "F'", "R", "R'" ]:
          self.operation(player_input)
          break
        else:
          print('        잘못된 입력입니다.')
      if self.cube.solved():
        return self.cube
    print(f'{self.num}번 플레이어의 차례가 끝났습니다.')
    return self.cube

In [19]:
while True:
  cube = Cube().input()
  if cube.getNum() not in depth:
    print('잘못된 입력입니다.')
  else:
    break

num_player = int(input('플레이어 수를 입력하세요 >> '))
point = int(input('플레이어가 가질 포인트를 입력하세요 >> '))
players = [ Player(i + 1, point) for i in range(num_player) ]

flag = False
while True:
  for player in players:
    cube = player.play(cube)
    if cube.solved():
      print(f'\n<< 승자는 {player.num}번 플레이어 입니다! >>')
      flag = True
      break
  if flag:
    break

윗   면 입력 >> W W Y B
앞   면 입력 >> W G Y G
오른 면 입력 >> O G B R
아랫 면 입력 >> R B W Y
뒷   면 입력 >> R O B R
왼   면 입력 >> G O Y O
플레이어 수를 입력하세요 >> 2
플레이어가 가질 포인트를 입력하세요 >> 100

    W W
    B Y
G O W G O G R O 
O Y G Y R B R B 
    R B
    Y W

1번 플레이어 차례 입니다.
주사위 눈은 5입니다.
    기회가 5번 남았습니다.
    회전하려면 기호를, 힌트를 보려면 H를 입력하세요 >> H
        정답까지 11 회전 남았습니다.
        포인트가 99점 남았습니다.
    회전하려면 기호를, 힌트를 보려면 H를 입력하세요 >> F
    기회가 4번 남았습니다.
    회전하려면 기호를, 힌트를 보려면 H를 입력하세요 >> H
        정답까지 10 회전 남았습니다.
        포인트가 98점 남았습니다.
    회전하려면 기호를, 힌트를 보려면 H를 입력하세요 >> F
    기회가 3번 남았습니다.
    회전하려면 기호를, 힌트를 보려면 H를 입력하세요 >> H
        정답까지 11 회전 남았습니다.
        포인트가 97점 남았습니다.
    회전하려면 기호를, 힌트를 보려면 H를 입력하세요 >> F'
    기회가 2번 남았습니다.
    회전하려면 기호를, 힌트를 보려면 H를 입력하세요 >> U'
    기회가 1번 남았습니다.
    회전하려면 기호를, 힌트를 보려면 H를 입력하세요 >> H
        정답까지 11 회전 남았습니다.
        포인트가 96점 남았습니다.
    회전하려면 기호를, 힌트를 보려면 H를 입력하세요 >> U
1번 플레이어의 차례가 끝났습니다.

    W W
    Y O
G R G W B G R O 
O B Y G Y B R B 
    R O
    Y W

2번 플레이어 차례 입니다.
주사위 눈은 5입니